# Линейная регрессия

In [11]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
# возьмем данные о домах в Калифорнии
from sklearn.datasets import fetch_california_housing

## Аналитическое решение, метод наименьших квадратов (МНК)

В функции `linear_model.LinearRegression()` используется аналитическое решение. \
Метод наименьших квадратов (МНК).

$$ \bar{w}=\left(X^{T} X\right)^{-1} X^{T} y=Q X^{T} y $$

> Теорема [Гаусса-Маркова](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%93%D0%B0%D1%83%D1%81%D1%81%D0%B0_%E2%80%94_%D0%9C%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D0%B0), говорит о том, \
что, если выполнены все условия теоремы, МНК всегда находит оптимальные оценки параметров.

Обращение матриц имеет кубическую сложность.\
При большом количестве признаков это процесс ресурсозатратный.

Второй недостаток МНК — это невозможность инкрементального обучения, \
или обучения в режиме реального времени.

Третий заключается в том, что матрица Q может не существовать.\
Это связано с математическими особенностями вычисления обратной матрицы.\
Причина этой проблемы — мультиколлинеарность факторов (сильная корреляционная связь). \
Из-за этого коэффициенты линейной регрессии становятся слишком большими \
и модель становится неустойчивой. \
Проблема решается с помощью регуляризации.

In [12]:
data = fetch_california_housing(as_frame=True)

housing_data = data['frame']
housing_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [13]:
# проверим, что данные числовые и нет пропусков
housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [14]:
# признаки
X = housing_data.drop(['MedHouseVal'], axis=1)
# целевой признак
y = housing_data['MedHouseVal']

# разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# создаём объект класса LinearRegression
linear_regression_model = linear_model.LinearRegression()

# обучаем модель
linear_regression_model.fit(X_train, y_train) 

# получаем предсказание
y_pred = linear_regression_model.predict(X_test)

# посчитаем метрику Mean Absolute Percentage Error (MAPE) 
# (показывает среднюю абсолютную процентную ошибку предсказанных значений в отношении фактических)
mape_linear_regression = metrics.mean_absolute_percentage_error(y_test, y_pred)

print('MAPE:', mape_linear_regression)

MAPE: 0.31978371646123604


В среднем модель ошибается на $\pm 32$%

In [15]:
# свободный член w0
print(f'w0: {np.round(linear_regression_model.intercept_, 5)}') 

# остальные параметры модели w1, w2, ..., wm
print(f'wi: {np.round(linear_regression_model.coef_, 5)}')

w0: -37.02783
wi: [ 0.4476   0.00957 -0.12476  0.79447 -0.      -0.00344 -0.41856 -0.43341]


## Численное решение

Для минимизации функции потерь используется метод **градиентного спуска (Gradient descent)**.

Для линейной регрессии необходимо найти такие параметры $w_0$, $w_1$, $w_2$, ... $w_m$, \
в которых находится минимум функции потерь \
(Функция потерь должна быть дифференцируема во всех точках в области определения. [Список доступных функций](https://scikit-learn.ru/1-5-stochastic-gradient-descent/#mathematical-formulation)).

[**Градиент**](https://ru.wikipedia.org/wiki/%D0%93%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82) — это вектор, 
который состоит из частных производных по параметрам функции.

$$ \nabla L(w)=\left(\frac{\partial L(w)}{\partial w_{0}}, \frac{\partial L(w)}{\partial w_{1}}, \frac{\partial L(w)}{\partial w_{2}}, \ldots, \frac{\partial L(w)}{\partial w_{m}}\right), $$

где — $L(w)$ функция потерь, зависящая от параметров модели, функция может быть любой (например, MSE). \
$\nabla$ — символ набла — символьное сокращение градиента, читается как «градиент функции $L(w)$». 

**Градиент** — это вектор, который показывает направление наискорейшего роста функции, \
а его длина — это само значение скорости функции в точке.

Если поставить перед градиентом знак минус $ - \nabla L(w)$, то мы получим вектор **антиградиента**, \
который показывает в сторону наискорейшего убывания функции потерь.

Следующая точка ищется по правилу

$$ w^{(k+1)} = w^{(k)} - \eta \nabla L(w^{(k)}), $$

где $w$ — это вектор параметров модели, \
а индекс в круглых скобках сверху означает номер точки в пространстве. 

Новая координата $w^{(k+1)}$ в пространстве параметров определяется как текущая координата $w^{(k)}$ \
минус скорость роста в текущей точке $\nabla L(w^{(k)})$, помноженная на коэффициент «скольжения».

$\eta$ (читается как «эта») - это поправочный коэффициент, который носит название темп обучения (**learning rate**).

Теоретически в точке минимума длина вектора равна 0, то есть движения не происходит. \
Это свойство используется в качестве критерия остановки алгоритма.

Одна из проблем градиентного спуска заключается в том, \
что алгоритм может «застрять» в локальном минимуме и не выйти из него.\
Для решения этой проблемы используются его [модификации](https://habr.com/ru/articles/413853/).

Применим реализацию стохастического градиентного спуска для линейной регрессии \
из библиотеки **sklearn** — [SGDRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html).

Перед применением необходимо нормализовать данные, иначе велика вероятность, что алгоритм не сойдется (не найдет минимум).

In [16]:
# инициализируем нормализатор MinMaxScaler
mm_scaler = preprocessing.MinMaxScaler()

# производим стандартизацию
mm_scaler.fit(X_train)

X_train_scaled = mm_scaler.transform(X_train)
X_test_scaled = mm_scaler.transform(X_test)

In [17]:
# создаём объект класса линейной регрессии с SGD
sgd_regressor_model = linear_model.SGDRegressor(random_state=42)

# обучаем модель — ищем параметры по методу SGD
sgd_regressor_model.fit(X_train_scaled, y_train)
 
# делаем предсказание
y_pred = sgd_regressor_model.predict(X_test_scaled)

# считаем MAPE
mape_sgd_regressor = metrics.mean_absolute_percentage_error(y_test, y_pred)

print('MAPE:', mape_sgd_regressor)

MAPE: 0.34322653785811025


In [18]:
# разница метрики MAPE с аналитическим решением 
mape_sgd_regressor - mape_linear_regression

0.023442821396874203

Метод градиентного спуска несколько хуже улавливает зависимости, чем аналитическое решение. \
Что ожидаемо.

In [19]:
# свободный член w0
print(f'w0: {np.round(sgd_regressor_model.intercept_, 5)}') 

# остальные параметры модели w1, w2, ..., wm
print(f'wi: {np.round(sgd_regressor_model.coef_, 5)}')

w0: [1.7095]
wi: [ 5.83848  0.76685  0.16545  0.11691  0.13821 -0.07374 -1.77815 -1.77282]


### Параметры SGDRegressor

Полный набор описан в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html).

- `loss` — функция потерь. \
По умолчанию используется `squared_loss` — это **MSE**. \
Но могут использоваться и другие. \
Например, `huber` определяет функцию потерь Хьюбера. \
Эта функция менее чувствительна к выбросам, чем **MSE**.

- `max_iter` — максимальное количество итераций, выделенное на сходимость. \
Значение по умолчанию — `1000`.

- `learning_rate` — режим управления темпом обучения. \
Значение по умолчанию — `invscaling`. Этот режим уменьшает темп обучения по формуле, $\eta_t = \frac{\eta_0}{t^p}$.\
Если нужно, чтобы темп обучения не менялся на протяжении всего обучения, можно выставить на `constant`.

- `eta0` — начальное значение темпа обучения $\eta_0$. \
Значение по умолчанию — 0.01.\
Если параметр `learning_rate="constant"`, \
то значение этого параметра будет темпом обучения на протяжении всех итераций.

- `power_t` — значение мощности уменьшения $p$ в формуле  $\eta_t = \frac{\eta_0}{t^p}$, \
где $\eta_0$ — начальное значение темпа обучения, \
а $t$ — номер итерации алгоритма. \
Данный параметр отвечает за степень знаменателя \
(чем больше степень, тем быстрее уменьшается значение темпа обучения с каждой итерацией). \
Значение по умолчанию — `0.25`.

Есть возможность дообучить модель на новых данных в режиме реального времени ([инкрементальное обучение](https://coderzcolumn.com/tutorials/machine-learning/scikit-learn-incremental-learning-for-large-datasets)). \
Повторный вызов `fit()` уточняет уже существующие параметры модели.